# 1. Datasets Requirement before Phenotyping
## 1.1 Find all admission items having Scan Records
### Neuroimaging/Vascularimaging
* DWI MRI / MRA: Gold standard in Neuroimaging.
* CT Scan: More available, maybe larger cohort, but not sensitive to early ischemic changes.
* Plus CTA: Angiography helps guide treatments, often preferred in emergency stroke settings

## 1.2 Or admission items by clinical phenotypes
### 1.2.1 Clinical Assessment
* NIHSS (National Institutes of Health Stroke Scale): Gold standard, assessment of consciousness, motor function, sensory function and language ability.
* Start from Large Vessel Occulusions (LOVs)

### 1.2.2 Stroke Risk Factors (Comorbidity)
* High blood pressure (typically > 180 mmHg)
* Atrial fibrillation
* cholesterol levels
* hyperlipidemia
* diabetes

## **Supporting Literature**:
* 'Diagnostic accuracy of clinical tools for assessment of acute stroke: a systematic review' https://bmcemergmed.biomedcentral.com/articles/10.1186/s12873-019-0262-1
* 'Comparative analysis, applications, and interpretation of electronic health record-based stroke phenotyping methods'
https://biodatamining.biomedcentral.com/articles/10.1186/s13040-020-00230-x

# 2. Cohort Extract Coding
## 2.1 Phenotyping by ICD-9 codes of stroke & NIHSS Clinical assessment
### 2.1.1 NIHSS Score
* Table: `chartevents`
* Itemid for NIHSS score: Search for this based on clinical charting of NIHSS scores or documented neurological exams in clinical notes.
* Criteria: Use NIHSS ≥ 6 to identify patients likely to have LVOs.

### 2.1.2 Clinical Notes of vital signs
* Table: `chartevents`
* Criteria: Use Natural Language Processing (NLP) to search for terms related to *aphasia*, *neglect*, and *hemiparesis* etc. in the clinical notes, particularly in discharge summaries or neurological exams.

## 2.2 Lab Results for Stroke Risk Factors (Weak)
* Table: `lab_event`
* **LDL cholesterol (50931)** and **HDL cholesterol (50924)** are markers of atherosclerosis.
**Blood glucose (50809)** and HbA1c levels are useful for identifying diabetic patients at higher stroke risk.

## 2.3 Diagnostics of Risk Factors (For features)
* Table: `diagnostic_icd`
* Criteria: ICD-10 codes like I48.0 for *Atrial fibrillation*, G45.9 for TIA, I10 for Hypertension, E11 for Type2 Diabetes and E78 for Hyperlipidemia.

## Environment setting and libraries import
* Install libraries required for BigQuery and data accessing

In [1]:
import pandas as pd
from pandas_gbq import read_gbq
import os
from google.cloud import bigquery

In [2]:
client = bigquery.Client(location="US")

* Link to my Google Service Account key so that the MIMIC database can be accessed via
BigQuery.

* By using Google Colab, the account can be easily authenciated by auth module.

## Query function for BigQuery
* Get ready for accessing data via query API
* Set the parameters required for BigQuery

In [3]:
# Set up environment variables
project_id = 'comp90089-440404'

# Read data from BigQuery into pandas dataframes
def run_query(query, project_id=project_id):
    return read_gbq(
        query,
        project_id=project_id,
        dialect='standard'
    )

## Extracting Stroke Patients in ICU

This SQL code identifies a group of patients who have been diagnosed with a stroke and were also admitted to the ICU. It specifically targets patients with ICD-9 codes related to stroke (433 and 434) and matches their records to ICU admissions, providing a list of these patients sorted by their unique identifiers.

In [4]:
# SQL cod for extracting patients with all criterion
query_isp=f"""
WITH stroke_patients AS (
  SELECT di.subject_id, di.hadm_id
  FROM physionet-data.mimiciv_hosp.diagnoses_icd di
  WHERE di.icd_version = 9 AND (di.icd_code LIKE '433%' OR di.icd_code LIKE '434%')
),

icu_stroke_patients AS (
  SELECT DISTINCT sp.subject_id, sp.hadm_id
  FROM stroke_patients sp
  JOIN `physionet-data.mimiciv_icu.icustays` AS icustays
    ON sp.subject_id = icustays.subject_id
    AND sp.hadm_id = icustays.hadm_id
)

SELECT subject_id, hadm_id
FROM icu_stroke_patients
ORDER BY subject_id, hadm_id
"""

In [5]:
# Execute the query and fetch results
df_isp = run_query(query_isp)

# Display results
print(df_isp.shape)

Downloading: 100%|██████████|
(2193, 2)


# 3. Feature Extracting
15 features from the following 5 aspects are considered:
* Demographics (**Age**, **Gender**, **Weight**, **Height**, **Infection**, **LoS**, **Admission**)
* ICU Vital Signs (**BP**, **Heart Rate**, **Temperature**, **Respiratory Rate** )
* Infections (**WBC Count**, **CRP Level**)
* Lab Cardiovascular Tests (**CK-MB Component**, **Troponin T Level**)
* Comorbidities (**Hypertension**, **Diabetes**, **Astrial Fibrillation**, **Charlson Index**)

In [6]:
# subject_id and admission_id lists
subject_ids = list(df_isp['subject_id'].unique())
cohort_subject_ids = "[" + ", ".join(str(id) for id in subject_ids) + "]"

hadm_ids=list(df_isp['hadm_id'].unique())
cohort_hadm_ids = "[" + ", ".join(str(id) for id in hadm_ids) + "]"

## 3.1 Demographics

## Extracting Age and Gender for ICU Stroke Patients

This retrieves age and gender details for ICU stroke patients, identified by `subject_ids`.


In [7]:
query_gender_age=f"""
WITH dmg_age AS (
SELECT subject_id, anchor_age AS age
FROM physionet-data.mimiciv_hosp.patients
WHERE subject_id IN UNNEST ({cohort_subject_ids})
),

dmg_gender AS (
SELECT subject_id,gender
FROM physionet-data.mimiciv_hosp.patients
WHERE subject_id IN UNNEST ({cohort_subject_ids})
)

SELECT dmg_age.subject_id, dmg_age.age, dmg_gender.gender
FROM dmg_age
LEFT JOIN dmg_gender ON dmg_age.subject_id = dmg_gender.subject_id
ORDER BY dmg_age.subject_id
"""

In [8]:
df_gender_age = run_query(query_gender_age)
df_isp_cohort=df_isp.merge(df_gender_age, on='subject_id', how='left')

Downloading: 100%|██████████|


### Demographics-2: Weight and Height (average)

* Itemid for Weight (Kg) in table `chartevents`: (226512(Admission Weight), 224639(Daily Weight))
* Itemid for Height (cm) in table `chartevents`:226730

In [9]:
query_wtht=f"""
WITH
dmg_weight AS (
SELECT ie.hadm_id,
  AVG(ce.valuenum) AS weight
FROM ({query_isp}) ie
-- admission weight
LEFT JOIN`physionet-data.mimiciv_icu.chartevents` As ce
  ON ie.hadm_id = ce.hadm_id AND itemid IN (226512, 224639)
WHERE ce.valuenum IS NOT NULL
GROUP BY ie.hadm_id
),

dmg_height AS (
SELECT  isp.hadm_id,
  MAX(ce.valuenum) AS height
FROM ({query_isp}) isp
-- admission height
LEFT JOIN`physionet-data.mimiciv_icu.chartevents` As ce
  ON isp.hadm_id = ce.hadm_id AND itemid = 226730
WHERE ce.valuenum IS NOT NULL
GROUP BY isp.hadm_id
)

SELECT dmg_weight.hadm_id, dmg_weight.weight, dmg_height.height
FROM dmg_weight
LEFT JOIN dmg_height ON dmg_weight.hadm_id = dmg_height.hadm_id
ORDER BY dmg_weight.hadm_id
"""

In [10]:
df_wtht = run_query(query_wtht)
df_isp_cohort=df_isp_cohort.merge(df_wtht, on='hadm_id', how='left')

Downloading: 100%|██████████|


### ICU Stay
* Length of Stay (sum_los)
* ICU Stay Counts (n_stays)

In [11]:
query_los=f"""
SELECT
  icustays.hadm_id As hadm_id,
  count(icustays.stay_id) As n_stays,
  sum(icustays.los) As sum_los
FROM
`physionet-data.mimiciv_icu.icustays` AS icustays
WHERE
  subject_id IN UNNEST ({cohort_subject_ids}) AND
  hadm_id IN UNNEST ({cohort_hadm_ids})
GROUP BY
 	hadm_id
ORDER BY
  hadm_id
"""

In [12]:
df_los = run_query(query_los)
df_isp_cohort=df_isp_cohort.merge(df_los, on='hadm_id', how='left')

Downloading: 100%|██████████|


## 3.2 ICU Vital Signs

### **ICU Vital Signs (Statistics)**
**References**:
* https://intjem.biomedcentral.com/articles/10.1186/1865-1380-5-3
* https://neurolrespract.biomedcentral.com/articles/10.1186/s42466-023-00238-8

In [13]:
query_vs = f"""

WITH vs_sbp AS (
SELECT hadm_id, MAX(valuenum) AS sbp_max
FROM physionet-data.mimiciv_icu.chartevents
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
itemid IN (
SELECT itemid
FROM physionet-data.mimiciv_icu.d_items
WHERE label LIKE '%Arterial Blood Pressure%'
)
GROUP BY hadm_id
),

vs_hr AS (
SELECT hadm_id, AVG(valuenum) AS heart_rate_avg
FROM physionet-data.mimiciv_icu.chartevents
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
itemid IN (
SELECT itemid
FROM physionet-data.mimiciv_icu.d_items
WHERE label LIKE '%Heart Rate%'
)
GROUP BY hadm_id
ORDER BY hadm_id
),

vs_rr AS (
SELECT  hadm_id, AVG(valuenum) AS respiratory_rate_avg
FROM physionet-data.mimiciv_icu.chartevents
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
itemid IN (
SELECT itemid
FROM physionet-data.mimiciv_icu.d_items
WHERE label LIKE '%Respiratory Rate%'
)
GROUP BY hadm_id
ORDER BY  hadm_id
),

vs_tmp AS(
SELECT hadm_id, MAX(valuenum) AS rec_temperature_max_F
FROM physionet-data.mimiciv_icu.chartevents
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
itemid IN (
SELECT itemid
FROM physionet-data.mimiciv_icu.d_items
WHERE label LIKE '%Temperature Fahrenheit%'
)
GROUP BY hadm_id
ORDER BY hadm_id
)

SELECT vs_rr.hadm_id, vs_tmp.rec_temperature_max_F, vs_sbp.sbp_max, vs_hr.heart_rate_avg, vs_rr.respiratory_rate_avg
FROM vs_rr
LEFT JOIN vs_tmp ON vs_tmp.hadm_id = vs_rr.hadm_id
LEFT JOIN vs_sbp ON vs_tmp.hadm_id = vs_sbp.hadm_id
LEFT JOIN vs_hr ON vs_tmp.hadm_id = vs_hr.hadm_id
ORDER BY vs_rr.hadm_id

"""

In [14]:
df_vs = run_query(query_vs)
df_isp_cohort=df_isp_cohort.merge(df_vs, on='hadm_id', how='left')

Downloading: 100%|██████████|


## 3.3 Inflammatory Infections

### **Infections Biomarkers**

### WBC Count (White Blood Cell Counts)
* Biomarker WBC counts consider all WBC lab test records for lacking number of a single WBC bomarker
* This biomarker might not be sufficient for the cohort

In [15]:
query_wbc = f"""
SELECT  hadm_id, MAX(valuenum) AS wbc_count
FROM physionet-data.mimiciv_hosp.labevents
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
itemid IN (
SELECT itemid
FROM physionet-data.mimiciv_hosp.d_labitems
WHERE LOWER(label) LIKE '%wbc%'
)
GROUP BY hadm_id
ORDER BY  hadm_id
"""

In [16]:
df_wbc = run_query(query_wbc)
df_isp_cohort=df_isp_cohort.merge(df_wbc, on='hadm_id', how='left')

Downloading: 100%|██████████|


In [17]:
query_lymphocyte = f"""
SELECT hadm_id, AVG(valuenum) AS lymphocyte_percentage
FROM physionet-data.mimiciv_hosp.labevents
WHERE hadm_id IN UNNEST({cohort_hadm_ids}) AND
      itemid IN (
          SELECT itemid
          FROM physionet-data.mimiciv_hosp.d_labitems
          WHERE LOWER(label) LIKE '%lymphocyte%'
      )
GROUP BY hadm_id
ORDER BY hadm_id
"""

In [18]:
df_lymphocyte = run_query(query_lymphocyte)
df_isp_cohort=df_isp_cohort.merge(df_lymphocyte, on='hadm_id', how='left')

Downloading: 100%|██████████|


### RBC Count (Red Blood Cell Counts)

In [19]:
query_rbc = f"""
SELECT  hadm_id, MAX(valuenum) AS rbc_count
FROM physionet-data.mimiciv_hosp.labevents
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
itemid IN (
SELECT itemid
FROM physionet-data.mimiciv_hosp.d_labitems
WHERE LOWER(label) LIKE '%rbc%'
)
GROUP BY hadm_id
ORDER BY  hadm_id
"""

In [20]:
df_rbc = run_query(query_rbc)
df_isp_cohort=df_isp_cohort.merge(df_rbc, on='hadm_id', how='left')

Downloading: 100%|██████████|


### CRP Levels (C-Reactive Protein)
* This biomarker records might be limited in terms of our cohort
* Substitution could be **Fibrinogen**, **Lactate**, **Albumin**
* Albumin and Fibrinogen might involve multiple subtypes of tests
**References**: https://doi.org/10.5853/jos.2018.02369

In [21]:
query_lactate = f"""
SELECT  hadm_id, MAX(valuenum) AS lactate_level_max
FROM physionet-data.mimiciv_hosp.labevents
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
itemid IN (
SELECT itemid
FROM physionet-data.mimiciv_hosp.d_labitems
WHERE LOWER(label) LIKE '%lactate%'
)
GROUP BY hadm_id
ORDER BY hadm_id
"""

In [22]:
df_lactate = run_query(query_lactate)
df_isp_cohort=df_isp_cohort.merge(df_lactate, on='hadm_id', how='left')

Downloading: 100%|██████████|


In [23]:
query_alb = f"""
SELECT hadm_id, MIN(valuenum) AS alb_level_min
FROM physionet-data.mimiciv_hosp.labevents
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
itemid IN (
SELECT itemid
FROM physionet-data.mimiciv_hosp.d_labitems
WHERE LOWER(label) LIKE '%albumin%'
)
GROUP BY hadm_id
ORDER BY hadm_id
"""

In [24]:
df_alb = run_query(query_alb)
df_isp_cohort=df_isp_cohort.merge(df_alb, on='hadm_id', how='left')

Downloading: 100%|██████████|


## 3.4 Lab Test Results

### Tropoin T Levels

In [25]:
query_trt = f"""
SELECT hadm_id, MAX(valuenum) AS trt_max
FROM physionet-data.mimiciv_hosp.labevents
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
itemid IN (
SELECT itemid
FROM physionet-data.mimiciv_hosp.d_labitems
WHERE LOWER(label) LIKE '%troponin t%'
)
GROUP BY hadm_id
ORDER BY hadm_id
"""

In [26]:
df_trt = run_query(query_trt)
df_isp_cohort=df_isp_cohort.merge(df_trt, on='hadm_id', how='left')

Downloading: 100%|██████████|


### Glucose Level (Hyperglycemia)
* Use count of abnormal glucose test results (very high or very low).

In [27]:
query_glucose = f"""
SELECT hadm_id,
       (SUM(CASE WHEN valuenum > 180 THEN 1 ELSE 0 END)+SUM(CASE WHEN valuenum < 70 THEN 1 ELSE 0 END)) AS glucose_extreme_count
FROM physionet-data.mimiciv_hosp.labevents
WHERE hadm_id IN UNNEST({cohort_hadm_ids}) AND
      itemid IN (
          SELECT itemid
          FROM physionet-data.mimiciv_hosp.d_labitems
          WHERE LOWER(label) LIKE '%glucose%'
      )
GROUP BY hadm_id
ORDER BY hadm_id
"""

In [28]:
# Run the query to get the glucose data
df_glucose = run_query(query_glucose)
df_isp_cohort=df_isp_cohort.merge(df_glucose, on='hadm_id', how='left')

Downloading: 100%|██████████|


### Sodium Level (Hypernatremia)
* Same statistic with Glucose

In [29]:
query_sodium = f"""
SELECT hadm_id,
       (SUM(CASE WHEN valuenum > 145 THEN 1 ELSE 0 END)+SUM(CASE WHEN valuenum < 135 THEN 1 ELSE 0 END)) AS sodium_extreme_count
FROM physionet-data.mimiciv_hosp.labevents
WHERE hadm_id IN UNNEST({cohort_hadm_ids}) AND
      itemid IN (
          SELECT itemid
          FROM physionet-data.mimiciv_hosp.d_labitems
          WHERE LOWER(label) LIKE '%sodium%'
      )
GROUP BY hadm_id
ORDER BY hadm_id
"""

In [30]:
# Run the query to get the sodium data
df_sodium = run_query(query_sodium)
df_isp_cohort=df_isp_cohort.merge(df_sodium, on='hadm_id', how='left')

Downloading: 100%|██████████|


## 3.5 Comorbidities

In [31]:
query_hpt = f"""
SELECT hadm_id,
CASE WHEN COUNT(icd_code) > 0 THEN 1 ELSE 0 END AS has_hypertension
FROM physionet-data.mimiciv_hosp.diagnoses_icd
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
icd_code IN (
SELECT icd_code
FROM physionet-data.mimiciv_hosp.d_icd_diagnoses
WHERE LOWER(long_title) LIKE '%hypertension%'
)
GROUP BY  hadm_id
ORDER BY  hadm_id
"""

In [32]:
df_hpt = run_query(query_hpt)
df_isp_cohort=df_isp_cohort.merge(df_hpt, on='hadm_id', how='left')
df_isp_cohort['has_hypertension'] = df_isp_cohort['has_hypertension'].fillna(0).astype(int)

Downloading: 100%|██████████|


In [33]:
query_dbt = f"""
SELECT hadm_id,
CASE WHEN COUNT(icd_code) > 0 THEN 1 ELSE 0 END AS has_diabetes
FROM physionet-data.mimiciv_hosp.diagnoses_icd
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
icd_code IN (
SELECT icd_code
FROM physionet-data.mimiciv_hosp.d_icd_diagnoses
WHERE LOWER(long_title) LIKE '%diabetes%' OR LOWER(long_title) LIKE '%type 2 diabetes%'
)
GROUP BY  hadm_id
ORDER BY  hadm_id
"""

In [34]:
df_dbt = run_query(query_dbt)
df_isp_cohort=df_isp_cohort.merge(df_dbt, on='hadm_id', how='left')
df_isp_cohort['has_diabetes'] = df_isp_cohort['has_diabetes'].fillna(0).astype(int)

Downloading: 100%|██████████|


### Atrial Fibrillation

In [35]:
query_af = f"""
SELECT hadm_id,
CASE WHEN COUNT(icd_code) > 0 THEN 1 ELSE 0 END AS has_af
FROM physionet-data.mimiciv_hosp.diagnoses_icd
WHERE hadm_id IN UNNEST ({cohort_hadm_ids}) AND
icd_code IN (
SELECT icd_code
FROM physionet-data.mimiciv_hosp.d_icd_diagnoses
WHERE LOWER(long_title) LIKE '%atrial fibrillation%' OR LOWER(long_title) LIKE '%atrial flutter%'
)
GROUP BY  hadm_id
ORDER BY  hadm_id
"""

In [36]:
df_af = run_query(query_af)
df_isp_cohort=df_isp_cohort.merge(df_af, on='hadm_id', how='left')
df_isp_cohort['has_af'] = df_isp_cohort['has_af'].fillna(0).astype(int)

Downloading: 100%|██████████|


### Comorbidity Score
* charlson_comorbidity_index

In [37]:
query_cs = f"""
WITH
comorbidity_weights AS (
    SELECT 'I10' AS icd_code, 1 AS weight  -- Hypertension
    UNION ALL
    SELECT 'I21', 1  -- Myocardial Infarction (ICD-10)
    UNION ALL
    SELECT '410', 1  -- Myocardial Infarction (ICD-9)
    UNION ALL
    SELECT 'I50', 2  -- Congestive Heart Failure (ICD-10)
    UNION ALL
    SELECT '428', 2  -- Congestive Heart Failure (ICD-9)
    UNION ALL
    SELECT 'E10', 1  -- Diabetes without complications (ICD-10)
    UNION ALL
    SELECT '250', 1  -- Diabetes without complications (ICD-9)
    UNION ALL
    SELECT 'E11', 2  -- Diabetes with complications (ICD-10)
    UNION ALL
    SELECT 'N18', 2  -- Chronic Kidney Disease (ICD-10)
    UNION ALL
    SELECT '585', 2  -- Chronic Kidney Disease (ICD-9)
    UNION ALL
    SELECT 'C34', 2  -- Malignant Neoplasm of Lung (ICD-10)
    UNION ALL
    SELECT '162', 2  -- Malignant Neoplasm of Lung (ICD-9)
    UNION ALL
    SELECT 'B20', 6  -- HIV/AIDS
    UNION ALL
    SELECT 'I25', 1  -- Coronary Artery Disease (ICD-10)
    UNION ALL
    SELECT '414', 1  -- Coronary Artery Disease (ICD-9)
    UNION ALL
    SELECT 'K70', 3  -- Liver Disease (ICD-10)
    UNION ALL
    SELECT 'K74', 3  -- Liver Disease (ICD-10)
    UNION ALL
    SELECT 'J44', 1  -- COPD (ICD-10)
    UNION ALL
    SELECT '496', 1  -- COPD (ICD-9)
    UNION ALL
    SELECT 'C50', 2  -- Malignant Neoplasm of Breast (ICD-10)
    UNION ALL
    SELECT '174', 2  -- Malignant Neoplasm of Breast (ICD-9)
    UNION ALL
    SELECT 'F01', 3  -- Dementia (ICD-10)
    UNION ALL
    SELECT 'F03', 3  -- Dementia (ICD-10)
    UNION ALL
    SELECT 'I48', 1  -- Atrial Fibrillation (ICD-10)
)

-- Calculate Charlson Comorbidity Index for each patient in the cohort
SELECT
       di.hadm_id,
       SUM(COALESCE(cw.weight, 0)) AS charlson_comorbidity_index
FROM physionet-data.mimiciv_hosp.diagnoses_icd di
LEFT JOIN comorbidity_weights cw
    ON di.icd_code = cw.icd_code
WHERE hadm_id IN UNNEST ({cohort_hadm_ids})
GROUP BY di.hadm_id
ORDER BY di.hadm_id;

"""

In [38]:
df_cs = run_query(query_cs)
df_isp_cohort=df_isp_cohort.merge(df_cs, on='hadm_id', how='left')

Downloading: 100%|██████████|


In [39]:
dataX=df_isp_cohort.copy()

## 3.6 Admission Type

### **Unique admission: Records with readmission**
* Treat each hadm_id as an indivdual (a sample), we get 6581 records stored in *full_cohort*, 1944 out of which get readmitted.

In [40]:
query_sp = f"""
WITH stroke_patients AS (
  SELECT DISTINCT di.subject_id, di.hadm_id
  FROM physionet-data.mimiciv_hosp.diagnoses_icd di
  WHERE di.icd_version = 9 
    AND (di.icd_code LIKE '433%' OR di.icd_code LIKE '434%')
)

SELECT sp.subject_id, sp.hadm_id
FROM stroke_patients sp
ORDER BY sp.subject_id, sp.hadm_id
"""

In [41]:
df_sp=run_query(query_sp)

Downloading: 100%|██████████|


In [42]:
# subject_id and admission_id lists
sp_subject_ids = list(df_sp['subject_id'].unique())
sp_cohort_subject_ids = "[" + ", ".join(str(id) for id in sp_subject_ids) + "]"
#print(len(subject_ids))

sp_hadm_ids=list(df_sp['hadm_id'].unique())
sp_cohort_hadm_ids = "[" + ", ".join(str(id) for id in sp_hadm_ids) + "]"

In [43]:
query_date=f"""
SELECT
    subject_id,
    hadm_id,
    admittime
FROM physionet-data.mimiciv_hosp.admissions
WHERE hadm_id IN UNNEST ({sp_cohort_hadm_ids});
"""

In [44]:
df_isp_date=run_query(query_date)

Downloading: 100%|██████████|


In [45]:
df_isp_date['admittime'] = pd.to_datetime(df_isp_date['admittime'])

merged_df = dataX.merge(df_isp_date, 
                        on=['subject_id', 'hadm_id'], 
                        how='left')

merged_df['discharge_date'] = merged_df['admittime'] + pd.to_timedelta(merged_df['sum_los'], unit='D')

readmissions = merged_df.merge(
    df_isp_date,
    on='subject_id',
    suffixes=('', '_next')
)

readmissions = readmissions[
    (readmissions['hadm_id'] != readmissions['hadm_id_next']) &
    (readmissions['admittime_next'] > readmissions['discharge_date']) &
    (readmissions['admittime_next'] <= readmissions['discharge_date'] + pd.Timedelta(days=365))
]

readmissions = readmissions[['subject_id', 'hadm_id']]

In [46]:
class1 = dataX.merge(readmissions[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'], how='inner')
class1['has_readmission'] = 1  # Set has_readmission to 1

In [47]:
no_readmission_subject_ids = df_sp['subject_id'].value_counts()[lambda x: x == 1].index
filtered_dataX_no_readmission = dataX[dataX['subject_id'].isin(no_readmission_subject_ids)]

In [48]:
filtered_dataX_no_readmission = filtered_dataX_no_readmission.copy()

# Calculate the number of null values in each row
filtered_dataX_no_readmission['null_count'] = filtered_dataX_no_readmission.isnull().sum(axis=1)

# Sort the DataFrame by 'null_count' in ascending order
df_least_nulls = filtered_dataX_no_readmission.sort_values(by='null_count')#.head(891)

# Drop the 'null_count' column if no longer needed
df_least_nulls = df_least_nulls.drop(columns=['null_count'])

# Create a copy for class0 and set 'has_readmission' to 0
class0 = df_least_nulls.copy()
class0['has_readmission'] = 0  # Set has_readmission to 0


In [49]:
data = pd.concat([class1, class0], ignore_index=True)